In [10]:
# IMPORTS

import pandas as pd
import h5py
import numpy as np
import sys
import matplotlib.pyplot as plt
import neurokit2 as nk
import random
import networkx as nx
import torch
from torch_geometric.datasets import KarateClub
from torch_geometric.utils import to_networkx # Visualization
import matplotlib.pyplot as plt
from torch.nn import Linear                   # Define layers
from torch_geometric.nn import GCNConv
from scipy.signal import savgol_filter
from scipy.ndimage import gaussian_filter1d 
import pywt # pip install PyWavelets
from scipy.signal import medfilt
import cv2 # pip install opencv-python  

In [11]:
# CARREGAR OS DADOS

def carregar_ecgs(unlabel, umdavb, rbbb, lbbb, sb, st, af, filtrado):

    caminho_arquivo = "../../Projeto/Database/exams.csv"
    dados = pd.read_csv(caminho_arquivo)
    arquivos_usados = ["exams_part0.hdf5", "exams_part1.hdf5",
                    "exams_part2.hdf5", "exams_part3.hdf5", "exams_par4.hdf5", "exams_part5.hdf5",
                    "exams_part6.hdf5", "exams_part7.hdf5", "exams_par8.hdf5", "exams_part9.hdf5",
                    "exams_part10.hdf5", "exams_part11.hdf5", "exams_part12.hdf5", "exams_part13.hdf5", 
                    "exams_part14.hdf5", "exams_part15.hdf5", "exams_part16.hdf5", "exams_part17.hdf5"]

    ecg_normal_linhas = dados.index[(dados.iloc[:, 14].isin(arquivos_usados)) &
                                    (dados.iloc[:, 4] == False) &
                                    (dados.iloc[:, 5] == False) &
                                    (dados.iloc[:, 6] == False) &
                                    (dados.iloc[:, 7] == False) &
                                    (dados.iloc[:, 8] == False) &
                                    (dados.iloc[:, 9] == False) ]
    
    ecg_umdavb_linhas = dados.index[(dados.iloc[:, 14].isin(arquivos_usados)) &
                                    (dados.iloc[:, 4] == True) &
                                    (dados.iloc[:, 5] == False) &
                                    (dados.iloc[:, 6] == False) &
                                    (dados.iloc[:, 7] == False) &
                                    (dados.iloc[:, 8] == False) &
                                    (dados.iloc[:, 9] == False) &
                                    (dados.iloc[:, 13] == False)]
    
    ecg_rbbb_linhas = dados.index[(dados.iloc[:, 14].isin(arquivos_usados)) &
                                    (dados.iloc[:, 4] == False) &
                                    (dados.iloc[:, 5] == True) &
                                    (dados.iloc[:, 6] == False) &
                                    (dados.iloc[:, 7] == False) &
                                    (dados.iloc[:, 8] == False) &
                                    (dados.iloc[:, 9] == False) &
                                    (dados.iloc[:, 13] == False)]
    
    ecg_lbbb_linhas = dados.index[(dados.iloc[:, 14].isin(arquivos_usados)) &
                                    (dados.iloc[:, 4] == False) &
                                    (dados.iloc[:, 5] == False) &
                                    (dados.iloc[:, 6] == True) &
                                    (dados.iloc[:, 7] == False) &
                                    (dados.iloc[:, 8] == False) &
                                    (dados.iloc[:, 9] == False) &
                                    (dados.iloc[:, 13] == False)]
    
    ecg_sb_linhas = dados.index[(dados.iloc[:, 14].isin(arquivos_usados)) &
                                    (dados.iloc[:, 4] == False) &
                                    (dados.iloc[:, 5] == False) &
                                    (dados.iloc[:, 6] == False) &
                                    (dados.iloc[:, 7] == True) &
                                    (dados.iloc[:, 8] == False) &
                                    (dados.iloc[:, 9] == False) &
                                    (dados.iloc[:, 13] == False)]
    
    ecg_st_linhas = dados.index[(dados.iloc[:, 14].isin(arquivos_usados)) &
                                    (dados.iloc[:, 4] == False) &
                                    (dados.iloc[:, 5] == False) &
                                    (dados.iloc[:, 6] == False) &
                                    (dados.iloc[:, 7] == False) &
                                    (dados.iloc[:, 8] == True) &
                                    (dados.iloc[:, 9] == False) &
                                    (dados.iloc[:, 13] == False)]
    
    ecg_af_linhas = dados.index[(dados.iloc[:, 14].isin(arquivos_usados)) &
                                    (dados.iloc[:, 4] == False) &
                                    (dados.iloc[:, 5] == False) &
                                    (dados.iloc[:, 6] == False) &
                                    (dados.iloc[:, 7] == False) &
                                    (dados.iloc[:, 8] == False) &
                                    (dados.iloc[:, 9] == True) &
                                    (dados.iloc[:, 13] == False)]



    print("Número de linhas ecg_normal_linhas:", len(ecg_normal_linhas))
    print("Número de linhas ecg_umdavb_linhas:", len(ecg_umdavb_linhas))
    print("Número de linhas ecg_rbbb_linhas:", len(ecg_rbbb_linhas))
    print("Número de linhas ecg_lbbb_linhas:", len(ecg_lbbb_linhas))
    print("Número de linhas ecg_sb_linhas:", len(ecg_sb_linhas))
    print("Número de linhas ecg_st_linhas:", len(ecg_st_linhas))
    print("Número de linhas ecg_af_linhas:", len(ecg_af_linhas))

    caminho_interferencias = "../../Projeto/Database/resultados_interferencia.csv"
    interferencias = pd.read_csv(caminho_interferencias)
    interferencias_ids = interferencias['exam_id'].tolist()

    ecg_normal_linhas = dados.index[~dados['exam_id'].isin(interferencias_ids) &
                                    (dados.iloc[:, 14].isin(arquivos_usados)) &
                                    (dados.iloc[:, 4] == False) &
                                    (dados.iloc[:, 5] == False) &
                                    (dados.iloc[:, 6] == False) &
                                    (dados.iloc[:, 7] == False) &
                                    (dados.iloc[:, 8] == False) &
                                    (dados.iloc[:, 9] == False) ]
    
    ecg_umdavb_linhas = dados.index[~dados['exam_id'].isin(interferencias_ids) &
                                    (dados.iloc[:, 14].isin(arquivos_usados)) &
                                    (dados.iloc[:, 4] == True) &
                                    (dados.iloc[:, 5] == False) &
                                    (dados.iloc[:, 6] == False) &
                                    (dados.iloc[:, 7] == False) &
                                    (dados.iloc[:, 8] == False) &
                                    (dados.iloc[:, 9] == False) &
                                    (dados.iloc[:, 13] == False)]
    
    ecg_rbbb_linhas = dados.index[~dados['exam_id'].isin(interferencias_ids) &
                                    (dados.iloc[:, 14].isin(arquivos_usados)) &
                                    (dados.iloc[:, 4] == False) &
                                    (dados.iloc[:, 5] == True) &
                                    (dados.iloc[:, 6] == False) &
                                    (dados.iloc[:, 7] == False) &
                                    (dados.iloc[:, 8] == False) &
                                    (dados.iloc[:, 9] == False) &
                                    (dados.iloc[:, 13] == False)]
    
    ecg_lbbb_linhas = dados.index[~dados['exam_id'].isin(interferencias_ids) &
                                    (dados.iloc[:, 14].isin(arquivos_usados)) &
                                    (dados.iloc[:, 4] == False) &
                                    (dados.iloc[:, 5] == False) &
                                    (dados.iloc[:, 6] == True) &
                                    (dados.iloc[:, 7] == False) &
                                    (dados.iloc[:, 8] == False) &
                                    (dados.iloc[:, 9] == False) &
                                    (dados.iloc[:, 13] == False)]
    
    ecg_sb_linhas = dados.index[~dados['exam_id'].isin(interferencias_ids) &
                                    (dados.iloc[:, 14].isin(arquivos_usados)) &
                                    (dados.iloc[:, 4] == False) &
                                    (dados.iloc[:, 5] == False) &
                                    (dados.iloc[:, 6] == False) &
                                    (dados.iloc[:, 7] == True) &
                                    (dados.iloc[:, 8] == False) &
                                    (dados.iloc[:, 9] == False) &
                                    (dados.iloc[:, 13] == False)]
    
    ecg_st_linhas = dados.index[~dados['exam_id'].isin(interferencias_ids) &
                                    (dados.iloc[:, 14].isin(arquivos_usados)) &
                                    (dados.iloc[:, 4] == False) &
                                    (dados.iloc[:, 5] == False) &
                                    (dados.iloc[:, 6] == False) &
                                    (dados.iloc[:, 7] == False) &
                                    (dados.iloc[:, 8] == True) &
                                    (dados.iloc[:, 9] == False) &
                                    (dados.iloc[:, 13] == False)]
    
    ecg_af_linhas = dados.index[~dados['exam_id'].isin(interferencias_ids) &
                                    (dados.iloc[:, 14].isin(arquivos_usados)) &
                                    (dados.iloc[:, 4] == False) &
                                    (dados.iloc[:, 5] == False) &
                                    (dados.iloc[:, 6] == False) &
                                    (dados.iloc[:, 7] == False) &
                                    (dados.iloc[:, 8] == False) &
                                    (dados.iloc[:, 9] == True) &
                                    (dados.iloc[:, 13] == False)]

    print("Tirando Interferência:")
    print("Número de linhas ecg_normal_linhas:", len(ecg_normal_linhas))
    print("Número de linhas ecg_umdavb_linhas:", len(ecg_umdavb_linhas))
    print("Número de linhas ecg_rbbb_linhas:", len(ecg_rbbb_linhas))
    print("Número de linhas ecg_lbbb_linhas:", len(ecg_lbbb_linhas))
    print("Número de linhas ecg_sb_linhas:", len(ecg_sb_linhas))
    print("Número de linhas ecg_st_linhas:", len(ecg_st_linhas))
    print("Número de linhas ecg_af_linhas:", len(ecg_af_linhas))

    ecg_normal_id = dados.iloc[ecg_normal_linhas, 0].tolist()
    ecg_umdavb_id = dados.iloc[ecg_umdavb_linhas, 0].tolist()
    ecg_rbbb_id = dados.iloc[ecg_rbbb_linhas, 0].tolist()
    ecg_lbbb_id = dados.iloc[ecg_lbbb_linhas, 0].tolist()
    ecg_sb_id = dados.iloc[ecg_sb_linhas, 0].tolist()
    ecg_st_id = dados.iloc[ecg_st_linhas, 0].tolist()
    ecg_af_id = dados.iloc[ecg_af_linhas, 0].tolist()

    random.seed(42) 

    ecg_normal_sample = random.sample(ecg_normal_id, unlabel) if len(ecg_normal_id) >= unlabel else ecg_normal_id
    ecg_umdavb_sample = random.sample(ecg_umdavb_id, umdavb) if len(ecg_umdavb_id) >= umdavb else ecg_umdavb_id
    ecg_rbbb_sample = random.sample(ecg_rbbb_id, rbbb) if len(ecg_rbbb_id) >= rbbb else ecg_rbbb_id
    ecg_lbbb_sample = random.sample(ecg_lbbb_id, lbbb) if len(ecg_lbbb_id) >= lbbb else ecg_lbbb_id
    ecg_sb_sample = random.sample(ecg_sb_id, sb) if len(ecg_sb_id) >= sb else ecg_sb_id
    ecg_st_sample = random.sample(ecg_st_id, st) if len(ecg_st_id) >= st else ecg_st_id
    ecg_af_sample = random.sample(ecg_af_id, af) if len(ecg_af_id) >= af else ecg_af_id

    ids_ecgs = ecg_normal_sample + ecg_umdavb_sample + ecg_rbbb_sample + ecg_lbbb_sample + ecg_sb_sample + ecg_st_sample + ecg_af_sample

    print("Número de ecgs pra usar:", len(ids_ecgs))

    
    if filtrado == True: arquivos_hdf5 = ["/scratch/guilherme.evangelista/Clustering-Paper/Projeto/Database/filtered_exams_0_1.hdf5",
                        "/scratch/guilherme.evangelista/Clustering-Paper/Projeto/Database/filtered_exams_2_3.hdf5",
                        "/scratch/guilherme.evangelista/Clustering-Paper/Projeto/Database/filtered_exams_4_5.hdf5",
                        "/scratch/guilherme.evangelista/Clustering-Paper/Projeto/Database/filtered_exams_6_7.hdf5",
                        "/scratch/guilherme.evangelista/Clustering-Paper/Projeto/Database/filtered_exams_8_9.hdf5",
                        "/scratch/guilherme.evangelista/Clustering-Paper/Projeto/Database/filtered_exams_10_11.hdf5",
                        "/scratch/guilherme.evangelista/Clustering-Paper/Projeto/Database/filtered_exams_12_13.hdf5",
                        "/scratch/guilherme.evangelista/Clustering-Paper/Projeto/Database/filtered_exams_14_15.hdf5",
                        "/scratch/guilherme.evangelista/Clustering-Paper/Projeto/Database/filtered_exams_16_17.hdf5"]
    
    else: arquivos_hdf5 = ['/scratch/pedro.bacelar/Clustering-Paper/Projeto/Database/exams_part0.hdf5',
                 '/scratch/pedro.bacelar/Clustering-Paper/Projeto/Database/exams_part1.hdf5',
                 '/scratch/pedro.bacelar/Clustering-Paper/Projeto/Database/exams_part2.hdf5',
                 '/scratch/pedro.bacelar/Clustering-Paper/Projeto/Database/exams_part3.hdf5',
                 '/scratch/pedro.bacelar/Clustering-Paper/Projeto/Database/exams_part4.hdf5',
                 '/scratch/pedro.bacelar/Clustering-Paper/Projeto/Database/exams_part5.hdf5',
                 '/scratch/pedro.bacelar/Clustering-Paper/Projeto/Database/exams_part6.hdf5',
                 '/scratch/pedro.bacelar/Clustering-Paper/Projeto/Database/exams_part7.hdf5',
                 '/scratch/pedro.bacelar/Clustering-Paper/Projeto/Database/exams_part8.hdf5',
                 '/scratch/pedro.bacelar/Clustering-Paper/Projeto/Database/exams_part9.hdf5',
                 '/scratch/pedro.bacelar/Clustering-Paper/Projeto/Database/exams_part10.hdf5',
                 '/scratch/pedro.bacelar/Clustering-Paper/Projeto/Database/exams_part11.hdf5',
                 '/scratch/pedro.bacelar/Clustering-Paper/Projeto/Database/exams_part12.hdf5',
                 '/scratch/pedro.bacelar/Clustering-Paper/Projeto/Database/exams_part13.hdf5',
                 '/scratch/pedro.bacelar/Clustering-Paper/Projeto/Database/exams_part14.hdf5',
                 '/scratch/pedro.bacelar/Clustering-Paper/Projeto/Database/exams_part15.hdf5',
                 '/scratch/pedro.bacelar/Clustering-Paper/Projeto/Database/exams_part16.hdf5',
                 '/scratch/pedro.bacelar/Clustering-Paper/Projeto/Database/exams_part17.hdf5']
        
    

    def get_ecg_data(file_path, exam_id):
        with h5py.File(file_path, 'r') as f:
            # Obter os IDs dos exames
            exam_ids = np.array(f['exam_id'])

            # Encontrar o índice correspondente ao exam_id de interesse
            exam_index = np.where(exam_ids == exam_id)[0]

            if len(exam_index) == 0:
                raise ValueError("Exam ID não encontrado.")
            else:
                exam_index = exam_index[0]
                # Acessar os tracings de ECG correspondentes ao exam_index
                exam_tracings = f['tracings'][exam_index]
                # Preencher tracings nulos com epsilon
                return exam_tracings

    exam_ids_to_cluster = ids_ecgs  # Substitua pelos IDs reais dos exames

    # Lista para armazenar todos os tracings de ECG
    all_tracings = []

    # Obter os tracings de ECG para cada exam_id e armazenar na lista
    for exam_id in exam_ids_to_cluster:
        found = False  # Sinalizador para verificar se o exame foi encontrado em algum arquivo
        for arquivo in arquivos_hdf5:
            try:
                tracings = get_ecg_data(arquivo, exam_id)
                if tracings is not None:
                    tracing_transposto = np.array(tracings).T
                    all_tracings.append(tracing_transposto)
                    found = True  # Sinalizador para indicar que o exame foi encontrado
                    break  # Se encontrou, não precisa continuar buscando nos outros arquivos
            except ValueError as e:
                i = 0
            except Exception as e:
                i = 0
        
        if not found:
            print(f"Erro: exame ID {exam_id} não encontrado em nenhum dos arquivos.")

    # Verifique o tamanho da lista all_tracings para garantir que os dados foram coletados corretamente
    print("Número de ecgs que eram pra ser processados:", len(ids_ecgs))
    print(f"Número total de traçados processados: {len(all_tracings)}")

    # X será um array com um único array dentro, contendo todos os números do tracings.T
    X = np.array(all_tracings)
    return X , ids_ecgs

In [12]:
import numpy as np

def get_my_dataset(X, unlabel=1,umdavb=1,rbbb=1,lbbb=1,sb=1,st=1,af=1,train_ratio=0.8):
    """
    Prepara o dataset de ECG para o treinamento.
    
    Parâmetros:
      - X: array de ECG (formato: [n_samples, n_channels, length])
      - os números de amostras por classe (deve coincidir com a ordem de concatenação na função carregar_ecgs)
      - train_ratio: proporção dos dados para treinamento
      
    Retorna:
      - X (possivelmente normalizado),
      - y: vetor de labels (0: normal, 1: umdavb, 2: rbbb, 3: lbbb, 4: sb, 5: st, 6: af)
      - train_idx: índices de treinamento
      - test_idx: índices de teste
    """
    
    total_samples = unlabel + umdavb + rbbb + lbbb + sb + st + af
    if X.shape[0] != total_samples:
        raise ValueError(f"O número de traçados em X ({X.shape[0]}) não corresponde à soma esperada ({total_samples}).")
    
    # Cria os labels de acordo com a ordem de concatenação
    y = np.array([0]*unlabel + 
                 [1]*umdavb + 
                 [2]*rbbb + 
                 [3]*lbbb + 
                 [4]*sb + 
                 [5]*st + 
                 [6]*af)
    
    # Aqui assumimos que a normalização é feita sobre o último eixo (tempo)
    X_norm = X.copy().astype(np.float32)
    for i in range(X_norm.shape[0]):
        # Evita divisão por zero
        mean_val = X_norm[i].mean()
        std_val = X_norm[i].std() if X_norm[i].std() != 0 else 1.0
        X_norm[i] = (X_norm[i] - mean_val) / std_val
    
    # Cria a divisão em treinamento e teste (shuffle dos índices)
    indices = np.arange(total_samples)
    np.random.shuffle(indices)
    split = int(train_ratio * total_samples)
    train_idx = indices[:split]
    test_idx = indices[split:]
    
    return X_norm, y, train_idx, test_idx


In [13]:
import os
import uuid
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
import torch.utils.data
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

class SimTSCTrainer:
    def __init__(self, device, logger):
        """
        device: Ex.: torch.device("cuda" if torch.cuda.is_available() else "cpu")
        logger: um objeto com método .log() para imprimir e/ou salvar logs.
        """
        self.device = device
        self.logger = logger
        
        # Diretório temporário, se necessário
        self.tmp_dir = 'tmp'
        if not os.path.exists(self.tmp_dir):
            os.makedirs(self.tmp_dir)


    def fit(self, model, X, y, train_idx, distances, K, alpha,
            test_idx=None, report_test=False, batch_size=128, epochs=300):
        """
        Treina o modelo SimTSC com base em:
         - X: Tensores de ECG (torch.Tensor ou np.array convertido para torch.Tensor).
         - y: Rótulos das classes.
         - train_idx: Índices das amostras de treinamento.
         - distances: Matriz de distâncias (float32) entre TODAS as amostras.
         - K, alpha: Hiperparâmetros usados na construção do grafo.
         - test_idx: Índices das amostras de teste (opcional).
         - report_test: Se True, faz avaliação no conjunto de teste ao fim de cada época.
         - batch_size: Tamanho do lote total (será dividido entre batch principal e "outros").
         - epochs: Número de épocas de treinamento.
         
        Retorna o modelo com parâmetros carregados no melhor checkpoint (em acurácia).
        """
        self.K = K
        self.alpha = alpha
        
        # Garante que X e y sejam tensores no device
        if not isinstance(X, torch.Tensor):
            X = torch.from_numpy(X)
        if not isinstance(y, torch.Tensor):
            y = torch.from_numpy(y)
        
        self.X = X.to(self.device)
        self.y = y.to(self.device)
        
        # distances deve ser np.array ou torch.Tensor; converte para Tensor float32
        if not torch.is_tensor(distances):
            distances = torch.from_numpy(distances.astype(np.float32))
        self.adj = distances  # Armazenamos para uso no teste

        # Divisão do batch em train_batch_size + other_batch_size
        # As "other" samples servem para manter o subgrafo coerente
        train_batch_size = min(batch_size // 2, len(train_idx))
        other_idx = np.array([i for i in range(len(X)) if i not in train_idx])
        other_batch_size = min(batch_size - train_batch_size, len(other_idx))
        
        # DataLoader para as amostras de treino
        train_dataset = Dataset(train_idx)
        train_loader = torch.utils.data.DataLoader(train_dataset,
                                                   batch_size=train_batch_size,
                                                   shuffle=True,
                                                   num_workers=1)
        
        # Caso queiramos relatar métricas de teste a cada epoch
        if report_test and test_idx is not None:
            test_batch_size = min(batch_size // 2, len(test_idx))
            other_idx_test = np.array([i for i in range(len(X)) if i not in test_idx])
            other_batch_size_test = min(batch_size - test_batch_size, len(other_idx_test))
            
            test_dataset = Dataset(test_idx)
            test_loader = torch.utils.data.DataLoader(test_dataset,
                                                      batch_size=test_batch_size,
                                                      shuffle=True,
                                                      num_workers=1)
        
        # Otimizador
        optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=4e-3)
        
        # Checkpoint
        file_path = os.path.join(self.tmp_dir, str(uuid.uuid4()))
        best_acc = 0.0
        
        # Loop de treinamento
        for epoch in range(epochs):
            model.train()
            epoch_losses = []
            
            # Loop pelos batches de treino
            for sampled_train_idx in train_loader:
                # Gera "other" para compor o subgrafo
                sampled_other_idx = np.random.choice(other_idx, other_batch_size, replace=False)
                idx = np.concatenate((sampled_train_idx, sampled_other_idx))
                
                # Monta tensores
                _X = self.X[idx]
                _y = self.y[sampled_train_idx]  # Rótulos apenas do batch principal
                _adj = self.adj[idx][:, idx].to(self.device)

                # Forward
                outputs = model(_X, _adj, K, alpha)
                # A parte de "outputs" referente ao batch principal é a [:len(sampled_train_idx)]
                loss = F.nll_loss(outputs[:len(sampled_train_idx)], _y)
                
                # Backprop
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
                epoch_losses.append(loss.item())
            
            # Avalia no final de cada época
            model.eval()
            
            # Precisamos calcular a acurácia no treino ou no test?
            # Aqui, calculamos no conjunto de treino (sem logging de F1, mas poderíamos).
            acc = self._compute_accuracy(
                model, self.X, self.y, self.adj, self.K, self.alpha,
                train_loader, other_idx, other_batch_size
            )
            
            # Verifica se essa é a melhor acurácia até agora
            if acc >= best_acc:
                best_acc = acc
                torch.save(model.state_dict(), file_path)

            # Se quisermos também relatar métricas no conjunto de teste
            if report_test and test_idx is not None:
                test_acc = self._compute_accuracy(
                    model, self.X, self.y, self.adj, self.K, self.alpha,
                    test_loader, other_idx_test, other_batch_size_test
                )
                # Podemos também computar F1 no teste
                test_f1 = self._compute_f1(
                    model, self.X, self.y, self.adj, self.K, self.alpha,
                    test_loader, other_idx_test, other_batch_size_test
                )
                
                mean_loss_epoch = np.mean(epoch_losses) if epoch_losses else 0.0
                self.logger.log(
                    f"[Epoch {epoch}] Loss Treino: {mean_loss_epoch:.4f} | "
                    f"Acurácia Treino: {acc:.4f} | Melhor Treino: {best_acc:.4f} | "
                    f"Acurácia Teste: {test_acc:.4f} | F1 Teste: {test_f1:.4f}"
                )
            else:
                mean_loss_epoch = np.mean(epoch_losses) if epoch_losses else 0.0
                self.logger.log(
                    f"[Epoch {epoch}] Loss Treino: {mean_loss_epoch:.4f} | "
                    f"Acurácia Treino: {acc:.4f} | Melhor Treino: {best_acc:.4f}"
                )
        
        # Carrega o melhor checkpoint
        model.load_state_dict(torch.load(file_path))
        model.eval()
        os.remove(file_path)
        
        return model


    def test(self, model, test_idx, batch_size=128):
        """
        Realiza o teste usando exatamente o mesmo esquema de subgrafo adotado no treinamento:
         - Cria batch de teste
         - Concatena com some "other_idx_test"
         - Calcula forward no subgrafo e extrai métricas apenas das amostras do batch.
         
        Retorna:
         - Um dicionário com:
             'loss': loss média no conjunto de teste
             'accuracy': acurácia final
             'f1_macro': F1 média
             'f1_per_class': array com F1 de cada classe
        """

        # Preparação: tam. do batch e "other" no teste
        test_batch_size = min(batch_size // 2, len(test_idx))
        other_idx_test = np.array([i for i in range(len(self.X)) if i not in test_idx])
        other_batch_size_test = min(batch_size - test_batch_size, len(other_idx_test))
        
        # DataLoader para o conjunto de teste
        test_dataset = Dataset(test_idx)
        test_loader = torch.utils.data.DataLoader(test_dataset, 
                                                 batch_size=test_batch_size, 
                                                 shuffle=True, 
                                                 num_workers=1)
        
        # Variáveis para acumular perda e computar métricas
        total_loss = 0.0
        total_count = 0
        all_preds = []
        all_labels = []
        
        # Modo eval
        model.eval()
        
        with torch.no_grad():
            for batch_indices in test_loader:
                # Seleciona "others"
                sampled_other_idx = np.random.choice(other_idx_test, 
                                                     other_batch_size_test, 
                                                     replace=False)
                combined_idx = np.concatenate((batch_indices, sampled_other_idx))
                
                # Monta tensores
                _X = self.X[combined_idx]
                _adj = self.adj[combined_idx][:, combined_idx].to(self.device)
                
                # Forward
                outputs = model(_X, _adj, self.K, self.alpha)
                
                # Parte principal do lote
                main_outputs = outputs[:len(batch_indices)]
                main_labels = self.y[batch_indices]
                
                # Loss
                batch_loss = F.nll_loss(main_outputs, main_labels, reduction='sum')
                total_loss += batch_loss.item()
                total_count += len(batch_indices)
                
                # Predições
                preds = main_outputs.argmax(dim=1).cpu().numpy()
                all_preds.extend(preds)
                all_labels.extend(main_labels.cpu().numpy())
        
        # Loss média
        mean_loss = total_loss / total_count if total_count > 0 else 0.0
        
        # Acurácia
        all_preds = np.array(all_preds)
        all_labels = np.array(all_labels)
        correct = (all_preds == all_labels).sum()
        accuracy = correct / total_count if total_count > 0 else 0.0
        
        # F1
        f1_per_class = f1_score(all_labels, all_preds, average=None)
        f1_macro = f1_score(all_labels, all_preds, average='macro')
        
        # Log, se quiser
        self.logger.log(f"[TEST] Loss: {mean_loss:.4f} | Acc: {accuracy:.4f} | "
                        f"F1(macro): {f1_macro:.4f} | F1 por classe: {f1_per_class}")
        
        return {
            'loss': mean_loss,
            'accuracy': accuracy,
            'f1_macro': f1_macro,
            'f1_per_class': f1_per_class
        }


    ###########################################################################
    # Funções auxiliares internas para cálculo de acurácia e F1 no fit
    ###########################################################################
    def _compute_accuracy(self, model, X, y, adj, K, alpha,
                         loader, other_idx, other_batch_size):
        """
        Computa a acurácia, reproduzindo o esquema: (batch + other_idx).
        """
        correct = 0
        total = 0
        with torch.no_grad():
            for batch_idx in loader:
                # Monta as amostras "externas"
                sampled_other_idx = np.random.choice(other_idx, other_batch_size, replace=False)
                combined_idx = np.concatenate((batch_idx, sampled_other_idx))
                _X = X[combined_idx]
                _y = y[batch_idx]
                _adj = adj[combined_idx][:, combined_idx].to(self.device)
                
                outputs = model(_X, _adj, K, alpha)
                main_outputs = outputs[:len(batch_idx)]
                preds = main_outputs.argmax(dim=1)
                
                correct += (preds == _y).sum().item()
                total += len(batch_idx)
        
        return correct / total if total > 0 else 0.0

    def _compute_f1(self, model, X, y, adj, K, alpha,
                    loader, other_idx, other_batch_size):
        """
        Computa o F1 (macro) usando o mesmo esquema de (batch + other_idx).
        """
        all_preds = []
        all_labels = []
        with torch.no_grad():
            for batch_idx in loader:
                # Mesma lógica de subgrafo
                sampled_other_idx = np.random.choice(other_idx, other_batch_size, replace=False)
                combined_idx = np.concatenate((batch_idx, sampled_other_idx))
                _X = X[combined_idx]
                _y = y[batch_idx]
                _adj = adj[combined_idx][:, combined_idx].to(self.device)
                
                outputs = model(_X, _adj, K, alpha)
                main_outputs = outputs[:len(batch_idx)]
                
                preds = main_outputs.argmax(dim=1).cpu().numpy()
                labels = _y.cpu().numpy()
                
                all_preds.extend(preds)
                all_labels.extend(labels)
        
        if len(all_preds) == 0:
            return 0.0
        
        return f1_score(all_labels, all_preds, average='macro')


class SimTSC(nn.Module):
    def __init__(self, input_size, nb_classes, num_layers=1, n_feature_maps=64, dropout=0.5):
        super(SimTSC, self).__init__()
        self.num_layers = num_layers
        self.block_1 = ResNetBlock(input_size, n_feature_maps)
        self.block_2 = ResNetBlock(n_feature_maps, n_feature_maps)
        self.block_3 = ResNetBlock(n_feature_maps, n_feature_maps)
        if self.num_layers == 1:
            self.gc1 = GraphConvolution(n_feature_maps, nb_classes)
        elif self.num_layers == 2:
            self.gc1 = GraphConvolution(n_feature_maps, n_feature_maps)
            self.gc2 = GraphConvolution(n_feature_maps, nb_classes)
            self.dropout = dropout
        elif self.num_layers == 3:
            self.gc1 = GraphConvolution(n_feature_maps, n_feature_maps)
            self.gc2 = GraphConvolution(n_feature_maps, n_feature_maps)
            self.gc3 = GraphConvolution(n_feature_maps, nb_classes)
            self.dropout = dropout

    def forward(self, x, adj, K, alpha):
        ranks = torch.argsort(adj, dim=1)
        sparse_index = [[], []]
        sparse_value = []
        for i in range(len(adj)):
            _sparse_value = []
            for j in ranks[i][:K]:
                sparse_index[0].append(i)
                sparse_index[1].append(j)
                _sparse_value.append(1/np.exp(alpha * adj[i][j].cpu().item()))
            _sparse_value = np.array(_sparse_value)
            _sparse_value /= _sparse_value.sum()
            sparse_value.extend(_sparse_value.tolist())
        sparse_index = torch.LongTensor(sparse_index)
        sparse_value = torch.FloatTensor(sparse_value)
        adj = torch.sparse.FloatTensor(sparse_index, sparse_value, adj.size())
        device = self.gc1.bias.device
        adj = adj.to(device)

        x = self.block_1(x)
        x = self.block_2(x)
        x = self.block_3(x)
        x = F.avg_pool1d(x, x.shape[-1]).squeeze()

        if self.num_layers == 1:
            x = self.gc1(x, adj)
        elif self.num_layers == 2:
            x = F.relu(self.gc1(x, adj))
            x = F.dropout(x, self.dropout, training=self.training)
            x = self.gc2(x, adj)
        elif self.num_layers == 3:
            x = F.relu(self.gc1(x, adj))
            x = F.dropout(x, self.dropout, training=self.training)
            x = F.relu(self.gc2(x, adj))
            x = F.dropout(x, self.dropout, training=self.training)
            x = self.gc3(x, adj)

        x = F.log_softmax(x, dim=1)
        return x

class GraphConvolution(Module):
    def __init__(self, in_features, out_features, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(0))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):
        support = torch.mm(input, self.weight)
        output = torch.spmm(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

class ResNetBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ResNetBlock, self).__init__()
        self.expand = True if in_channels < out_channels else False
        self.conv_x = nn.Conv1d(in_channels, out_channels, 7, padding=3)
        self.bn_x = nn.BatchNorm1d(out_channels)
        self.conv_y = nn.Conv1d(out_channels, out_channels, 5, padding=2)
        self.bn_y = nn.BatchNorm1d(out_channels)
        self.conv_z = nn.Conv1d(out_channels, out_channels, 3, padding=1)
        self.bn_z = nn.BatchNorm1d(out_channels)
        if self.expand:
            self.shortcut_y = nn.Conv1d(in_channels, out_channels, 1)
        self.bn_shortcut_y = nn.BatchNorm1d(out_channels)

    def forward(self, x):
        out = F.relu(self.bn_x(self.conv_x(x)))
        out = F.relu(self.bn_y(self.conv_y(out)))
        out = self.bn_z(self.conv_z(out))
        if self.expand:
            x = self.shortcut_y(x)
        x = self.bn_shortcut_y(x)
        out += x
        out = F.relu(out)
        return out

class Dataset(torch.utils.data.Dataset):
    def __init__(self, idx):
        self.idx = idx

    def __getitem__(self, index):
        return self.idx[index]

    def __len__(self):
        return len(self.idx)


In [14]:
# 1) Carregar e preparar dados
X, ids_ecgs = carregar_ecgs(
    unlabel=15000, umdavb=2500, rbbb=2500,
    lbbb=2500, sb=2500, st=2500, af=2500,
    filtrado=True
)

# Aqui fazemos a separação e normalização (exemplo com 80% train, 20% test)
X_norm, y, train_idx, test_idx = get_my_dataset(
    X, 
    unlabel=15000, umdavb=2500, rbbb=2500,
    lbbb=2500, sb=2500, st=2500, af=2500,
    train_ratio=0.8
)
print("X_norm shape:", X_norm.shape)
print("Tamanho do y:", len(y))
print("Número de amostras em train_idx:", len(train_idx))
print("Número de amostras em test_idx:", len(test_idx))

Número de linhas ecg_normal_linhas: 272407
Número de linhas ecg_umdavb_linhas: 3735
Número de linhas ecg_rbbb_linhas: 6808
Número de linhas ecg_lbbb_linhas: 4176
Número de linhas ecg_sb_linhas: 4300
Número de linhas ecg_st_linhas: 6146
Número de linhas ecg_af_linhas: 4964
Tirando Interferência:
Número de linhas ecg_normal_linhas: 252167
Número de linhas ecg_umdavb_linhas: 3651
Número de linhas ecg_rbbb_linhas: 6703
Número de linhas ecg_lbbb_linhas: 4122
Número de linhas ecg_sb_linhas: 4248
Número de linhas ecg_st_linhas: 6038
Número de linhas ecg_af_linhas: 4804
Número de ecgs pra usar: 30000
Número de ecgs que eram pra ser processados: 30000
Número total de traçados processados: 30000
X_norm shape: (30000, 12, 4096)
Tamanho do y: 30000
Número de amostras em train_idx: 24000
Número de amostras em test_idx: 6000


In [ ]:
import os
import numpy as np
import torch
import time
from tqdm import tqdm
from dtaidistance import dtw
from joblib import Parallel, delayed
from sklearn.metrics import f1_score
import torch.nn.functional as F

# 1) Função otimizada para calcular DTW na CPU
def get_dtw_for_ecg_optimized(X, r=100):
    """
    Calcula a matriz de distâncias DTW de forma otimizada na CPU usando `dtw.distance_matrix_fast()`.
    
    Parâmetros:
        X (numpy array): Dados de entrada (shape: [n amostras, canais, comprimento da série]).
        r (int): Janela de restrição para o cálculo DTW.

    Retorna:
        distances (numpy array): Matriz de distâncias DTW simétrica.
    """
    # Garante que os dados estão no formato correto e mutáveis
    X_single = np.asarray(X[:, 0, :], dtype=np.float64)
    
    print("Calculando matriz DTW otimizada na CPU...")
    start_time = time.time()

    # Usa a função otimizada sem precisar de paralelização manual
    distances = dtw.distance_matrix_fast(X_single, window=r, parallel=True)

    elapsed_time = time.time() - start_time
    print(f"\nCálculo DTW finalizado em {elapsed_time / 60:.2f} minutos.")

    return distances

# Executa o cálculo da matriz DTW
distances = get_dtw_for_ecg_optimized(X_norm, r=100)
np.save("distances_dtw_otimizado.npy", distances)
print("Matriz de distâncias calculada com shape:", distances.shape)

# 2) Criar o modelo SimTSC
num_classes = len(np.unique(y))  
input_size = X_norm.shape[1]    

model = SimTSC(
    input_size=input_size,
    nb_classes=num_classes,
    num_layers=1,      
    n_feature_maps=64,  
    dropout=0.5
)

# 3) Definir o device e um logger
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print("Usando device:", device)

class Logger:
    def __init__(self, f):
        self.f = f
    def log(self, content):
        print(content)
        self.f.write(content + '\n')
        self.f.flush()

log_path = "./logs/simtsc_ecg_log.txt"
os.makedirs(os.path.dirname(log_path), exist_ok=True)
f = open(log_path, 'w')
logger = Logger(f)

# 4) Criar o trainer
trainer = SimTSCTrainer(device=device, logger=logger)

# 5) Treinar o modelo
K = 3
alpha = 0.3
epochs = 50
print("Iniciando treinamento...")

model = trainer.fit(
    model=model,
    X=X_norm,
    y=y,
    train_idx=train_idx,
    distances=distances,
    K=K,
    alpha=alpha,
    test_idx=test_idx,
    report_test=True,   
    batch_size=128,
    epochs=epochs
)

# 6) Avaliar o modelo no conjunto de teste final
class_names = ["normal", "1dAVb", "RBBB", "LBBB", "SB", "ST", "AF"]

print("\n--- Avaliação final no conjunto de teste ---")
results = trainer.test(model, test_idx, batch_size=128)
print("\nResultados finais:")
print("Loss no teste:", results['loss'])
print("Acurácia no teste:", results['accuracy'])
print("F1 Macro:", results['f1_macro'])
print("F1 por classe:")
for class_name, f1_val in zip(class_names, results['f1_per_class']):
    print(f"  {class_name}: {f1_val:.4f}")

# Fecha o logger
f.close()
